# Perform a BoM query

Both impacted substances queries and compliance queries can be performed on an XML BoM instead of a list
of Granta MI records. The BoM must be in the Granta 17/11 format or Granta 23/01 format. This example shows how to use
the ``lxml`` package with the XSD XML schema file to validate the XML format.

For help on constructing an XML BoM, consult the
[Creating a Bill of Materials from an external data source](3-5_Creating_a_Bill_of_Materials.py) example.

If the XML file is generated by a Granta MI product and has not been modified, it is possible to skip this step before
submitting the query. However, it is strongly advised to validate the XML BoM in all situations to avoid unexpected
server-side failures. If an invalid XML file is used in a query, an exception is raised by the ``requests`` HTTP
package, but it does not contain information about why the XML is non-compliant. A more detailed log is
reported on the server in the MI Service Layer log.

The following supporting files are required for this example:

* [bom-complex.xml](supporting-files/bom-complex.xml)
* [invalid-bom.xml](supporting-files/invalid-bom.xml)

The XSD XML schema is included in the library in the ``schemas`` sub-package.

## Validating an XML file with an XSD schema

The ``lxml`` package provides a similar API to the standard library xml module, but it includes some additional
functionality, including schema validation. First import the ``lxml`` package and then build a simple validator
function that takes both the XML file and schema file and returns the result.

In [ ]:
from lxml import etree


def xml_validator(xml: str, schema_file: str) -> bool:
    schema = etree.XMLSchema(file=schema_file)
    doc = etree.fromstring(xml)
    return schema.validate(doc)


# You can now use this function to test the validity of a BoM generated by BoM Analyzer.

In [ ]:
from ansys.grantami.bomanalytics.schemas import bom_schema_1711

valid_xml_file = "supporting-files/bom-complex.xml"
with open(valid_xml_file) as f:
    valid_xml = f.read()
result = xml_validator(valid_xml, bom_schema_1711)
result

You can also test a BoM that is valid XML but is not compliant with the schema.

In [ ]:
invalid_xml_file = "supporting-files/invalid-bom.xml"
with open(invalid_xml_file) as f:
    invalid_xml = f.read()
result = xml_validator(invalid_xml, bom_schema_1711)
result

## Run an impacted substances XML-based query

Now that you have validated the XML, you can build your XML BoM-based query.
First, connect to Granta MI.

In [ ]:
from ansys.grantami.bomanalytics import Connection

server_url = "http://my_grantami_server/mi_servicelayer"
cxn = Connection(server_url).with_credentials("user_name", "password").connect()

The impacted substances BoM query behaves similar to other impacted substances queries. However,
a BoM query can only accept a single BoM at a time, and so you only ever receive a single list
of substances impacted by the specified legislations.

In [ ]:
from ansys.grantami.bomanalytics import queries

SIN_LIST = "SINList"
impacted_substances_query = (
    queries.BomImpactedSubstancesQuery()
    .with_bom(valid_xml)
    .with_legislation_ids([SIN_LIST])
)

In [ ]:
impacted_substances_result = cxn.run(impacted_substances_query)
impacted_substances_result

The ``BomImpactedSubstancesQueryResult`` object returned after running the query for impacted substances now behaves
similarly to the result object for any other query for impacted substances. For example, you can print all substances
impacted by the legislation using an approach from the previous example.

In [ ]:
from tabulate import tabulate

rows = [
    [substance.cas_number, substance.max_percentage_amount_in_material]
    for substance in impacted_substances_result.impacted_substances
]
print(f'Substances impacted by "{SIN_LIST}" in a BoM (5/{len(rows)})')
print(tabulate(rows[:5], headers=["CAS Number", "Amount (wt. %)"]))

## Running a compliance XML-based query

Running a BoM compliance query produces the same result as if you had stored the BoM structure as linked **Products
and Parts** records in Granta MI.

In [ ]:
from ansys.grantami.bomanalytics import indicators

svhc = indicators.WatchListIndicator(
    name="SVHC",
    legislation_ids=["Candidate_AnnexXV"],
    default_threshold_percentage=0.1,
)
compliance_query = (
    queries.BomComplianceQuery()
    .with_bom(valid_xml)
    .with_indicators([svhc])
)

In [ ]:
compliance_result = cxn.run(compliance_query)
compliance_result

The ``BomComplianceQueryResult`` object returned after running the compliance query contains a list of
``PartWithComplianceResult`` objects, the behavior of which has been covered in a previous example.
The following cell prints the compliance status of the BoM.

In [ ]:
root_part = compliance_result.compliance_by_part_and_indicator[0]
print(f"BoM Compliance Status: {root_part.indicators['SVHC'].flag.name}")

## Invalid BoM exception

If you were to try the same query with an invalid BoM, you would see a stack trace informing you that the MI Service
Layer responded with a 500 HTTP response code. To raise the exception, change the constant ``RUN_QUERY``below to
``True``.

In [ ]:
broken_query = (
    queries.BomImpactedSubstancesQuery()
    .with_bom(invalid_xml)
    .with_legislation_ids([SIN_LIST])
)

RUN_QUERY = False

if RUN_QUERY:
    cxn.run(broken_query)